In [1]:
!pip install duckdb

import duckdb
import pandas as pd

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#con = duckdb.connect(database='/content/drive/MyDrive/projecteADSDB/explotation/explotation.duckdb', read_only=False)

con = duckdb.connect(database='./explotation/explotation.duckdb', read_only=False)

df_governance = pd.read_sql("SELECT * FROM Governance_Data", con)

# melt the datarame

In [4]:
# provisional treatment
df_governance.drop_duplicates(inplace=True)

In [5]:
aruba = df_governance[df_governance["Country Code"] == 'ABW']
aruba.head(20)

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],2000 [YR2000],2004 [YR2004],2008 [YR2008],2012 [YR2012],2016 [YR2016],2020 [YR2020]


In [6]:
df_governance.head(20)

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],2000 [YR2000],2004 [YR2004],2008 [YR2008],2012 [YR2012],2016 [YR2016],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.271724,-1.350647,-1.672096,-1.430373,-1.534049,-1.480538
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2.000000,2.000000,5.000000,7.000000,10.000000,10.000000,9.000000
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,4.787234,6.403941,0.485437,1.421801,3.846154,4.807693
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.419355,30.851065,15.270936,4.368932,11.848341,9.615385,10.576923
5,Afghanistan,AFG,Control of Corruption: Standard Error,CC.STD.ERR,0.340507,0.346906,0.213201,0.224345,0.186088,0.164448,0.157702
6,Afghanistan,AFG,Government Effectiveness: Estimate,GE.EST,-2.175167,-2.173946,-0.909889,-1.533435,-1.376419,-1.254182,-1.585145
7,Afghanistan,AFG,Government Effectiveness: Number of Sources,GE.NO.SRC,1.000000,1.000000,4.000000,6.000000,8.000000,8.000000,7.000000
8,Afghanistan,AFG,Government Effectiveness: Percentile Rank,GE.PER.RNK,0.000000,0.000000,18.905472,3.398058,5.687204,8.173077,5.288462
9,Afghanistan,AFG,"Government Effectiveness: Percentile Rank, Low...",GE.PER.RNK.LOWER,0.000000,0.000000,5.970149,0.485437,0.473934,4.807693,1.442308


In [7]:
df_governance.dropna(inplace=True)

#drop irrelevant columns
df_governance.drop(columns=['Country Name', 'Series Name'], inplace=True)
df_governance.rename(columns={'1996 [YR1996]': '1996', '2000 [YR2000]': '2000', '2004 [YR2004]':'2004', '2008 [YR2008]':'2008', '2012 [YR2012]':'2012', '2016 [YR2016]':'2016', '2020 [YR2020]':'2020'}, inplace=True)


df_governance = pd.melt(df_governance, id_vars=['Country Code', 'Series Code'], value_vars=['1996', '2000', '2004', '2008', '2012', '2016', '2020'], var_name='year', value_name='value')
df_governance['year']=df_governance['year'].astype(int)

#remove all indicators that are not estimate values
df_governance.drop(df_governance[~df_governance['Series Code'].str.contains("EST", case=False)].index, inplace=True)


aaaa = df_governance.groupby(['Country Code', 'year', 'Series Code'], as_index = False).count()
aaaa = aaaa[aaaa.value>1]

for cc in set(aaaa["Country Code"].tolist()):
  df_governance = df_governance[df_governance["Country Code"] != cc]


df_governance = df_governance.pivot(index=['Country Code', 'year'], columns= 'Series Code',values="value")

df_governance.reset_index(inplace=True)

df_governance.columns = df_governance.columns.str.rstrip('.EST')

In [8]:
df_governance

Series Code,Country Code,year,CC,G,PV,RL,RQ,VA
0,AFG,1996,-1.291705,-2.175167,-2.417310,-1.788075,-2.090330,-1.908540
1,AFG,2000,-1.271724,-2.173946,-2.438969,-1.780661,-2.080253,-2.031417
2,AFG,2004,-1.350647,-0.909889,-2.295682,-1.693925,-1.511549,-1.203034
3,AFG,2008,-1.672096,-1.533435,-2.691361,-1.903308,-1.607167,-1.168902
4,AFG,2012,-1.430373,-1.376419,-2.418561,-1.651893,-1.192580,-1.267430
...,...,...,...,...,...,...,...,...
1402,ZWE,2004,-1.253563,-1.001513,-1.204704,-1.688897,-1.974932,-1.553577
1403,ZWE,2008,-1.348838,-1.542653,-1.212338,-1.791505,-2.064896,-1.549031
1404,ZWE,2012,-1.381803,-1.375217,-0.778662,-1.630184,-1.871891,-1.471968
1405,ZWE,2016,-1.271258,-1.209890,-0.619938,-1.381370,-1.684896,-1.178460


# Store the dataframe

In [9]:
con.execute("DROP TABLE IF EXISTS Governance_Data;")
con.execute("CREATE TABLE IF NOT EXISTS Governance_Data AS SELECT * FROM df_governance")

close connection

In [10]:
con.close()